<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/test_cu_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test cu_cat for cuda version issues


In [1]:
# !pip install Ipython --upgrade

# %load_ext autoreload
# %autoreload 2

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==23.12.00 cudf-cu12==23.12.00 #cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 393.9/955.2 MB 27.7 MB/s eta 0:00:21

In [ ]:
!pip install --upgrade cu-cat
!pip install git+https://github.com/graphistry/pygraphistry.git@dev/depman_gpufeat


In [ ]:
import cu_cat
import cudf

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
# import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [ ]:
import graphistry

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__


In [ ]:
!nvidia-smi #-h #--format=csv
# !pip install nvsmi
# import nvsmi
# gpu=str(nvsmi.get_gpu_processes()).split('|')[3].split(':')[1]

## look at dataset (select non-sparse columns)

In [ ]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogsA=pd.read_parquet('part.88.parquet')


# Graphistry[AI] - Feature Engineering:
## CUDA_cat GPU engine
- data: **5million row** x dozens of sparse columns Windows log dataset
- data subset: parred down to **20k** rows, selecting just dense columns

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
winlogs=winlogsA[['LogonID','UserName','LogHost','Time','DomainName','LogonType','SubjectLogonID','Status','Destination','ServiceName']]#.convert_dtypes() #.replace('nan',np.nan).fillna('0o0o0')
winlogs=winlogs.sample(10000,replace=False)

In [ ]:
aa=cu_cat.TableVectorizer()
aa.fit_transform((winlogs))

In [ ]:
aa.fit(winlogs).transform((winlogs))

In [ ]:
g1 = graphistry.nodes(cudf.from_pandas(winlogs))

t=time()
g11=g1.umap(feature_engine='cu_cat',engine='cuml',memoize=False)
t2=time()-t
print(t2)

In [ ]:
g1 = graphistry.nodes((winlogs))

t=time()
g11=g1.umap(feature_engine='cu_cat',engine='cuml',memoize=False)
t2=time()-t
print(t2)